<a href="https://colab.research.google.com/github/Nashi8/Prompt-Engineering/blob/main/PE_ReAct.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
!pip install --quiet openai

from google.colab import userdata
from openai import OpenAI

api_key = userdata.get("118A_OpenAI_API")

if api_key is None:
    raise ValueError("Secret '118A_OpenAI_API' not found. "
                     "Go to Colab Settings → Secrets and add it.")

# Initialize OpenAI client
client = OpenAI(api_key=api_key)

print("OpenAI client initialized.")

OpenAI client initialized.


In [10]:
def call_llm(
    system_prompt: str,
    user_prompt: str,
    model: str = "gpt-4.1-mini",
    temperature: float = 0.3,
) -> str:
    """
    Thin wrapper around the OpenAI chat completion API.
    Returns the model's text output as a string.
    """
    response = client.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt},
        ],
        temperature=temperature,
    )
    return response.choices[0].message.content.strip()

In [6]:
import re

def generate_python_with_react(task_description: str):
    system_prompt = "You are an expert Python programmer who reasons step by step."
    user_prompt = f"""
You will first reason about the task, then output Python code.

TASK:
{task_description}

FORMAT YOUR RESPONSE LIKE THIS:

REASONING:
<your step-by-step reasoning>

CODE:
<begin_python_code>
# python code here
<end_python_code>
"""
    return call_llm(system_prompt, user_prompt)

def extract_code(react_output: str):
    start = "<begin_python_code>"
    end = "<end_python_code>"
    if start not in react_output or end not in react_output:
        raise ValueError("Could not find python code markers in the response.")
    return react_output.split(start)[1].split(end)[0].strip()

def run_code(code: str):
    exec_globals = {}
    exec_locals = {}
    exec(code, exec_globals, exec_locals)
    return exec_globals, exec_locals

In [ ]:
print("ReACT Python Code Assistant")
print("Describe a coding task. Type 'quit' to stop.\n")
print("Please complete at least 3 back-and-forth interactions.\n")

while True:
    user_task = input("You: ")
    if user_task.strip().lower() in {"quit", "exit"}:
        print("Assistant: Goodbye! Happy coding!")
        break

    react_output = generate_python_with_react(user_task)

    print("\nAssistant (reasoning + code):\n")
    print(react_output)
    print()

In [16]:
tasks = [
    "Write a Python function that checks if a number is prime.",
    "Modify the prime function to return False for numbers less than 2.",
    "Add a main block that tests the function on the numbers 1–10."
]

print("=== Scripted 3-Turn ReACT Code Conversation ===\n")

for t in tasks:
    print("You:", t)
    react_output = generate_python_with_react(t)
    print("Assistant (reasoning + code):\n")
    print(react_output)
    print()

=== Scripted 3-Turn ReACT Code Conversation ===

You: Write a Python function that checks if a number is prime.
Assistant (reasoning + code):

REASONING:
To check if a number is prime, we need to verify that it is greater than 1 and that it has no divisors other than 1 and itself. The straightforward way is to check divisibility from 2 up to the square root of the number because if a number n is divisible by some number greater than its square root, then it must also be divisible by some number smaller than its square root.

Steps:
1. If the number is less than or equal to 1, return False because primes are greater than 1.
2. Check divisibility from 2 up to and including the integer part of the square root of the number.
3. If any divisor is found, return False.
4. If no divisors are found, return True.

This approach is efficient enough for moderate-sized numbers.

CODE:
<begin_python_code>
import math

def is_prime(n):
    if n <= 1:
        return False
    max_divisor = int(math.is